In [1]:
from build123d import *
from ocp_vscode import *

In [2]:
import os

stp_fld = "../data/Traceparts-dataset/Step_models"
stp_files = []

for fld, sub_flds, files in os.walk(stp_fld):
    for fn in files:
        if fn.endswith(".stp"):
            stp_files.append(os.path.join(fld, fn))
print(f"Found {len(stp_files)} .stp files")

Found 600 .stp files


In [14]:
stp_sample = import_step("../data/Traceparts-dataset/Step_models/0/0_2.stp")
show(stp_sample)

c


In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
file_path.parent

PosixPath('/home/sjj/wa/AI-projects/agentic-eda/3D_STEP_Classification/data/Traceparts-dataset/Step_models/1')

In [19]:
folder_name, file_name

(PosixPath('/home/sjj/wa/AI-projects/agentic-eda/3D_STEP_Classification/data/Traceparts-dataset/Step_models/1'),
 '1_0.stp')

In [ ]:
import os
import os.path as osp
from pathlib import Path
import sys
sys.path.append(".")

import Graph_convertion.Parser as Parser
import Graph_convertion.Graphh as Graphh
from Graph_convertion.Graphh.utils import replace_nodes
from Graph_convertion.Parser.Make_graph import all_nodes_to_graph
import networkx as nx

name = "dummy"
graph_saves_paths = "/tmp/dummy_graph.graphml"
file_path = Path("/home/sjj/wa/AI-projects/agentic-eda/docs-papers/eda/封装EDA开发相关资料收集_0917/1_数据库资料/现有封装类型产品的POD和POD_3D图纸/POD_3D/CX-3D-PA0013/34mm-模型.stp")
folder_name = file_path.parent
file_name = file_path.name

headers, datas = Parser.parse_file(file_name, folder_name)
all_flat_nodes, fast_dict_search = Graphh.get_nodes_from_datas(datas)
print("   All nodes obtained")
replace_nodes(all_flat_nodes, fast_dict_search)
print("   All edges obtained")
G_simplex_d = nx.DiGraph()
all_nodes_to_graph(G_simplex_d, all_flat_nodes, name, graph_saves_paths)

   Number of composed id: 693
   All nodes obtained
   All edges obtained
   Graphh of dummy model realizing time: 0.5223867893218994 seconds


In [26]:
datas[10]

'#11=PRODUCT_DEFINITION(UNKNOWN,,#10,#7)'

In [16]:
fast_dict_search.keys()

dict_keys([])

In [17]:
from Graph_convertion.Graphh.Node import FlatNode

def remove_first_last_space(string):
    fixed_string = string
    if fixed_string[0] == " ":
        fixed_string = fixed_string[1:]
    if fixed_string[-1] == " ":
        fixed_string = fixed_string[:-1]
    return fixed_string


def split_recursive(string, father_diff_par):
    arguments = []
    arg = ''
    offset = 0
    off = 0
    diff_num_parentesis = string.count(')') - string.count('(') - 1
    for idx in range(len(string)):
        current_idx = idx + offset
        if current_idx >= len(string):
            break
        char = string[current_idx]
        if char != '(' and char != "," and char != ')':
            arg += char
            off += 1
        elif char == ',':
            arguments.append(arg)
            off += 1
            arg = ''
        elif char == '(':
            nau, new_offset = split_recursive(string[current_idx + 1:], diff_num_parentesis)
            offset += new_offset
            off += new_offset + 1
            arg = nau
        elif char == ')':
            if diff_num_parentesis > father_diff_par:
                arguments.append(arg)
                break
            else:
                diff_num_parentesis -= 1
                arguments.append(arg)

    return arguments, off + 1
def split_composed_arguments(string):
    parentesis_count = 0
    composed_arguments = []
    start = 0
    end = 0
    for idx, char1 in enumerate(string):
        if char1 == '(':
            parentesis_count += 1
        elif char1 == ')':
            parentesis_count -= 1
            if parentesis_count == 0:
                end = idx + 1
                comp_arg = string[start:end]
                comp_arg = remove_first_last_space(comp_arg)
                composed_arguments.append(comp_arg)
                start = idx + 1
    return composed_arguments

In [19]:
fast_dict_search = {}
all_flat_nodes = []
    
    
sample_line = '#110=(BOUNDED_SURFACE()B_SPLINE_SURFACE(3,2,((#98,#99,#100),(#101,#102,#103),(#104,#105,#106),(#107,#108,#109)),.UNSPECIFIED.,.F.,.F.,.U.)B_SPLINE_SURFACE_WITH_KNOTS((4,4),(3,3),(0.,1.),(0.,100.),.UNSPECIFIED.)GEOMETRIC_REPRESENTATION_ITEM()RATIONAL_B_SPLINE_SURFACE(((1.,0.70725583395941,1.),(1.,0.70727492527993,1.),(1.,0.70729401470976,1.),(1.,0.70731310181993,1.)))REPRESENTATION_ITEM('')SURFACE())'

id_type_arguments = sample_line.split("=")
id = id_type_arguments[0]
id = remove_first_last_space(id)

type_arguments = id_type_arguments[1]
type_arguments = remove_first_last_space(type_arguments)

composed_id = 0

if type_arguments[0] != '(':  # Normal elements
    type_arguments = type_arguments.split("(", 1)
    type = type_arguments[0]
    arguments = type_arguments[1]
    arguments, _ = split_recursive(arguments, 0)
else:  # Composed elements
    multiple_obj = type_arguments[1:][:-1]
    multiple_obj = split_composed_arguments(multiple_obj)
    arguments = []
    for i, m in enumerate(multiple_obj):
        m_type_arguments = m.split('(', 1)
        m_type = m_type_arguments[0]
        m_arguments = m_type_arguments[1]
        m_arguments, _ = split_recursive(m_arguments, 0)
        m_id = '##' + str(composed_id)  # TODO fa casino??? --> no
        composed_id += 1
        # node = GenericNode(m_id, m_type, m_arguments)
        flat_node = FlatNode(m_id, m_type, m_arguments)
        # all_nodes.append(node)
        all_flat_nodes.append(flat_node)
        fast_dict_search[m_id] = flat_node
        # Composed object properties
        if i == 0:
            type = 'COMPOSED_' + m_type
        arguments.append(m_id)

In [20]:
type_arguments, type, arguments, id

('(BOUNDED_SURFACE()B_SPLINE_SURFACE(3,2,((#98,#99,#100),(#101,#102,#103),(#104,#105,#106),(#107,#108,#109)),.UNSPECIFIED.,.F.,.F.,.U.)B_SPLINE_SURFACE_WITH_KNOTS((4,4),(3,3),(0.,1.),(0.,100.),.UNSPECIFIED.)GEOMETRIC_REPRESENTATION_ITEM()RATIONAL_B_SPLINE_SURFACE(((1.,0.70725583395941,1.),(1.,0.70727492527993,1.),(1.,0.70729401470976,1.),(1.,0.70731310181993,1.)))REPRESENTATION_ITEM()SURFACE())',
 'COMPOSED_BOUNDED_SURFACE',
 ['##0', '##1', '##2', '##3', '##4', '##5', '##6'],
 '#110')

In [22]:
flat_node

In [21]:
fast_dict_search, all_flat_nodes

({'##0': <Graph_convertion.Graphh.Node.FlatNode at 0x7684883fad80>,
  '##1': <Graph_convertion.Graphh.Node.FlatNode at 0x7684883f9370>,
  '##2': <Graph_convertion.Graphh.Node.FlatNode at 0x7684883f9040>,
  '##3': <Graph_convertion.Graphh.Node.FlatNode at 0x7684883fbd10>,
  '##4': <Graph_convertion.Graphh.Node.FlatNode at 0x7684883fb710>,
  '##5': <Graph_convertion.Graphh.Node.FlatNode at 0x7684883fb1a0>,
  '##6': <Graph_convertion.Graphh.Node.FlatNode at 0x7684883fb2f0>},
  <Graph_convertion.Graphh.Node.FlatNode at 0x7684883fb2f0>])

In [18]:
with open(file_path, encoding='latin-1') as f:
    data = f.read()

In [7]:
stps = []

stp_fld = "/home/sjj/wa/AI-projects/agentic-eda/docs-papers/eda/封装EDA开发相关资料收集_0917/1_数据库资料"
for fld, sub_folds, files in os.walk(stp_fld):
    for fn in files:
        if fn.endswith(".stp"):
            stps.append(os.path.join(fld, fn))

In [9]:
len(stps)

35

In [10]:
import shutil

target_fld = "/home/sjj/wa/AI-projects/agentic-eda/3D_STEP_Classification/data/cx_data"

for fn in stps:
    base_name = osp.basename(fn)
    target_fn = osp.join(target_fld, base_name)
    if osp.exists(target_fn):
        target_fn = target_fn.replace(".stp", "_dup.stp")
    shutil.copy(fn, target_fn)

In [11]:
from bs4 import BeautifulSoup

with open("/home/sjj/wa/AI-projects/agentic-eda/3D_STEP_Classification/docs/figures/Cylinder203r.svg", "r") as f:
    svg_content = f.read()

soup = BeautifulSoup(svg_content, 'xml') # Use 'xml' parser for SVG files

text_elements = soup.find_all('text')

stp_entities = list(set([text.get_text() for text in text_elements]))
len(stp_entities)

35

In [36]:
import json

with open('../docs/stp_entities.json', 'w') as f:
    json.dump(sorted(set(result)), f, indent=2)


In [37]:
import re
from itertools import chain

m = re.compile(r"([A-Z_0-9]+)\(.*\)")

result = list(chain.from_iterable(m.findall(d) for d in datas if d.strip()))

In [40]:
len(set(result))

61

In [41]:
result

['LENGTH_UNIT',
 'NAMED_UNIT',
 'NAMED_UNIT',
 'UNCERTAINTY_MEASURE_WITH_UNIT',
 'GEOMETRIC_REPRESENTATION_CONTEXT',
 'APPLICATION_CONTEXT',
 'DESIGN_CONTEXT',
 'MECHANICAL_CONTEXT',
 'PRODUCT',
 'PRODUCT_DEFINITION_FORMATION_WITH_SPECIFIED_SOURCE',
 'PRODUCT_DEFINITION',
 'PRODUCT_DEFINITION_SHAPE',
 'PRODUCT_RELATED_PRODUCT_CATEGORY',
 'CARTESIAN_POINT',
 'DIRECTION',
 'DIRECTION',
 'AXIS2_PLACEMENT_3D',
 'APPLICATION_CONTEXT',
 'MECHANICAL_CONTEXT',
 'PRODUCT',
 'PRODUCT_RELATED_PRODUCT_CATEGORY',
 'PRODUCT_DEFINITION_FORMATION_WITH_SPECIFIED_SOURCE',
 'APPLICATION_CONTEXT',
 'APPLICATION_PROTOCOL_DEFINITION',
 'DESIGN_CONTEXT',
 'PRODUCT_DEFINITION',
 'PRODUCT_DEFINITION_SHAPE',
 'SHAPE_REPRESENTATION',
 'SHAPE_DEFINITION_REPRESENTATION',
 'CARTESIAN_POINT',
 'DIRECTION',
 'DIRECTION',
 'AXIS2_PLACEMENT_3D',
 'APPLICATION_CONTEXT',
 'MECHANICAL_CONTEXT',
 'PRODUCT',
 'PRODUCT_RELATED_PRODUCT_CATEGORY',
 'PRODUCT_DEFINITION_FORMATION_WITH_SPECIFIED_SOURCE',
 'APPLICATION_CONTEXT',
 